In [0]:
#########################################################################################################################
#                                        COVID-19 DATA COLLECTOR FOR NETHERLANDS                                        #
# This is a part of a project. The collector scraps data via BS4 from RIVM and ECDC's websites, wrangle and manipulate  #
# the data convert them to a dataframe and save it as csv file. The csv files refresh themselves through main function  #
# every certain time. So,they will keep up to date data. April 2020  github/IjzerenSteen                                #
#########################################################################################################################

#Importing relevant library 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import threading
import time
from datetime import date, datetime, timedelta

#For using Google Drive path & Saving data to Google Drive 
from google.colab import drive
drive.mount('drive')

def main():
    t = threading.Timer(21600, main) #21600 - 6 hours
    t.start() 
    rivm_data1()
    rivm_data2()
    rivm_data3()
    rivm_data4()
    ecdc_data()
    print('1')
    
def rivm_data1():
    '''Total Cases and Deaths Numbers
    via RIVM.NL website'''
    
    #Scrapping the data via BS4
    request_data1=requests.get('https://www.rivm.nl/coronavirus-kaart-van-nederland').text
    soup1 = BeautifulSoup(request_data1,'html.parser')
    my_table1=soup1.find_all('table')
    
    #Scrapping the data via BS4
    request_data1=requests.get('https://www.rivm.nl/coronavirus-kaart-van-nederland').text
    soup1 = BeautifulSoup(request_data1,'html.parser')
    my_table1=soup1.find_all('table')
    
    #Setting the data to a dataframe
    my_table1[0]=str(my_table1[0]).replace('*','')
    my_table1[0]=str(my_table1[0]).replace('\n','')
    my_table1[0]=str(my_table1[0]).replace(')','')
    my_table1[0]=str(my_table1[0]).replace('(','')
    my_table1[0]=str(my_table1[0]).replace('+','')
    my_table1[0]=str(my_table1[0]).replace('.','')    
    df1= pd.read_html(str(my_table1[0]), index_col=False, header=None)[0]
    df1=df1.T
    df1.drop(df1.index[0],inplace=True)

    #Cleaning the data
    headers=["People tested positive", "Admitted to Hospital", "Death"]
    df1.columns=headers

    df1["People tested positive daily increase"]=df1["People tested positive"][2]
    df1["Admitted to Hospital daily increase"]=df1["Admitted to Hospital"][2]
    df1["Death daily increase"]=df1["Death"][2]
    df1.drop(2,0,inplace=True)

    df1["People tested positive"]= pd.to_numeric(df1["People tested positive"], downcast='integer')
    df1["Admitted to Hospital"] = pd.to_numeric(df1["Admitted to Hospital"], downcast='integer')
    df1["Death"] = pd.to_numeric(df1["Death"], downcast='integer')
    df1["People tested  daily increase"]= pd.to_numeric(df1["People tested positive daily increase"], downcast='integer')
    df1["Admitted to Hospital daily increase"] = pd.to_numeric(df1["Admitted to Hospital daily increase"], downcast='integer')
    df1["Death daily increase"] = pd.to_numeric(df1["Death daily increase"], downcast='integer')

    #Seave it to a csv file to a certain path
    df1.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/rivm_data_1_total.csv',  index = True, header=True)

def rivm_data2():
    '''Daily data of Hospitalized and Reported patients,
    and Death Numbers through Rivm.nl website'''
    
    # Scrapping the 3 csv data from rivm.nl
    request_data=requests.get('https://www.rivm.nl/coronavirus-covid-19/grafieken').text
    soup = BeautifulSoup(request_data,'html.parser')
    my_table=soup.find_all('script', {'type':"application/json"})
    my_table=str(my_table)
    my_table=my_table.split("csv")

    #Dataset1
    #Data cleaning 
    table_1 = my_table[4]
    table_1 = table_1.replace('\\u0022', '')
    table_1 = table_1.split("data:")[0]
    table_1 = str(table_1).split(']","pre')[0]
    table_1 = str(table_1).split("ren],")[1]
    table_1 = table_1.replace("]", '')
    table_1 = table_1.split("[")

    #Setting the data to a data frame 
    df1 = pd.DataFrame([sub.split(",") for sub in table_1 ])
    df1.drop([3], axis=1, inplace=True)
    df1.drop(0,inplace=True)
    df1.columns=["day_month","Reported_New_Cases","Reported_Previous_Cases"]

    #Changing the numerical columns from string to integer and adding a total column 
    df1.Reported_New_Cases = pd.to_numeric(df1.Reported_New_Cases, downcast='integer')
    df1.Reported_Previous_Cases = pd.to_numeric(df1.Reported_Previous_Cases, downcast='integer')
    df1.loc[:,"Reported_Total_Cases"] = df1.Reported_New_Cases + df1.Reported_Previous_Cases

    #Setting the date data to date form of pandas library
    df1['Year']= '2020'
    df1['date']=df1.day_month+' '+ df1.Year
    df1.date=df1.date.str.replace("mrt","mar")
    df1.date=df1.date.str.replace("mei","may")
    df1.date=pd.to_datetime(df1.date).dt.date
    df1.drop(['day_month'], axis=1, inplace=True)
    df1.drop(['Year'], axis=1, inplace=True)
    df1 = df1[['date', 'Reported_New_Cases', 'Reported_Previous_Cases', 'Reported_Total_Cases']]
    #df1.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/data_rivm_reported.csv',  index = True, header=True)

    #Dataset2
    #Data cleaning process 
    table_2 = my_table[3]
    table_2 = table_2.replace('\\u0022', '')
    table_2 = table_2.split("data:")[0]
    table_2 = table_2.split(']","pre')[0]
    table_2 = table_2.split("ren],")[1]
    table_2 = table_2.replace("]", '')
    table_2 = table_2.split("[")
    
    #Setting the data to a data frame 
    df2 = pd.DataFrame([sub.split(",") for sub in table_2 ])
    df2.drop([3], axis=1, inplace=True)
    df2.drop(0,inplace=True)
    df2.columns=["day_month","Hospitalized_New_Cases","Hospitalized_Previous_Cases"]
    
    #Changing the numerical columns from string to integer and adding a total column 
    df2.Hospitalized_New_Cases = pd.to_numeric(df2.Hospitalized_New_Cases, downcast='integer')
    df2.Hospitalized_Previous_Cases = pd.to_numeric(df2.Hospitalized_Previous_Cases, downcast='integer')
    df2.loc[:,"Hospitalized_Total_Cases"] = df2.Hospitalized_New_Cases + df2.Hospitalized_Previous_Cases

    #Setting the date data to date form of pandas library
    df2['Year']= '2020'
    df2['date']=df2.day_month+' '+ df2.Year
    df2.date=df2.date.str.replace("mrt","mar")
    df2.date=df2.date.str.replace("mei","may")
    df2.date=pd.to_datetime(df2.date).dt.date
    df2.drop(['day_month'], axis=1, inplace=True)
    df2.drop(['Year'], axis=1, inplace=True)
    df2 = df2[['date', 'Hospitalized_New_Cases', 'Hospitalized_Previous_Cases', 'Hospitalized_Total_Cases']]
    #df2.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/data_rivm_hospitalized.csv',  index = True, header=True)

    #Dataset3
    #Data cleaning process 
    table_3 = my_table[2]
    table_3 = table_3.replace('\\u0022', '')
    table_3 = table_3.split("data:")[0]
    table_3 = str(table_3).split(']","pre')[0]
    table_3 = str(table_3).split("ren],")[1]
    table_3 = table_3.replace("]", '')
    table_3 = table_3.split("[")

    #Setting the data to a data frame 
    df3 = pd.DataFrame([sub.split(",") for sub in table_3 ])
    df3.drop([3], axis=1, inplace=True)
    df3.drop(0,inplace=True)
    df3.columns=["day_month","New_Cases","Previous_Cases"]

    #Setting the date data to date form for pandas library
    df3.New_Cases = pd.to_numeric(df3.New_Cases, downcast='integer')
    df3.Previous_Cases = pd.to_numeric(df3.Previous_Cases, downcast='integer')
    df3.loc[:,"Deaths"] = df3.New_Cases + df3.Previous_Cases

    #Setting the date data to date form of pandas library
    df3['Year']= '2020'
    df3['date']=df3.day_month+' '+ df3.Year
    df3.date=df3.date.str.replace("mrt","mar")
    df3.date=df3.date.str.replace("mei","may")
    df3.date=pd.to_datetime(df3.date).dt.date
    df3.drop(['day_month'], axis=1, inplace=True)
    df3.drop(['Year'], axis=1, inplace=True)
    df3 = df3[['date', 'New_Cases', 'Previous_Cases', 'Deaths']]
    #df3.to_csv (r'data_rivm_death.csv',  index = True, header=True)
    
    #Merging 3 datasets to a data frame
    df_merged0=pd.merge(df1,df2[['date', 'Hospitalized_New_Cases', 'Hospitalized_Previous_Cases', 'Hospitalized_Total_Cases']], on='date', how="left") #Left join
    df_merged=pd.merge(df_merged0,df3[['date', 'New_Cases', 'Previous_Cases', 'Deaths']], on='date', how="left") #Left join
    df_merged['Country']='Netherlands'
    
    #Save the dataframe as csv file to a certain path
    df_merged.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/rivm_data_2_reported_hospitalized_deaths.csv',  index = True, header=True)

def rivm_data3():
    """PART 1 - The numbers of cases by municipality through rivm.nl
    PART 2 - The data of provinces from wikipedia
    PART 3 - Merging in a dataframe of both datasets"""

    #Part-1
    #Scrapping the data from rivm.nl 
    request_data=requests.get('https://www.rivm.nl/coronavirus-kaart-van-nederland').text
    soup = BeautifulSoup(request_data,'html.parser')
    my_table=soup.find_all('div', {'id':"csvData"})
    table=str(my_table[0]).split('\n')

    #Setting it to a dataframe 
    df2 = pd.DataFrame([sub.split(";") for sub in table ])

    #Cleaning the data 
    df2.drop(0,0,inplace=True)
    df2.drop(1,0,inplace=True)
    df2.dropna(inplace=True)
    df2.drop(df2.tail(0).index,0,inplace=True)
    df2.drop([0,6,7,8], 1, inplace=True)
    headers=["Municipality","Reported_Case_Number","Hospitalized_Case_Number","Death","Population"]
    df2.columns=headers
    df2['Population'] = pd.to_numeric(df2['Population'], downcast='integer')
    df2['Reported_Case_Number'] = pd.to_numeric(df2['Reported_Case_Number'], downcast='integer')
    df2['Hospitalized_Case_Number'] = pd.to_numeric(df2['Hospitalized_Case_Number'], downcast='integer')
    df2['Death'] = pd.to_numeric(df2['Death'], downcast='integer')

    #PART-2 
    #Scrapping the data from wikipedia
    request_data2 = requests.get('https://simple.wikipedia.org/wiki/List_of_municipalities_of_the_Netherlands').text
    soup2 = BeautifulSoup(request_data2,'html.parser')
    my_table2 = soup2.find_all('table')
    
    #Setting it to a dataframe
    df3 = pd.read_html(str(my_table2))[1]

    #PART-3
    #Merging the both datasets in a dataframe 
    df=pd.merge(df2,df3[['Municipality','Province']], on='Municipality', how="left") #Left join
    
    #Filling the null provinces & Cleaning the data
    df.loc[df['Municipality']=='West Betuwe', 'Province']='Gelderland'
    df.loc[df['Municipality']=='Westerkwartier', 'Province']='Groningen'
    df.loc[df['Municipality']=='Vijfheerenlanden', 'Province']='Utrecht'
    df.loc[df['Municipality']=='s-Gravenhage', 'Province']='South Holland'
    df.loc[df['Municipality']=='Noardeast-Fryslân', 'Province']='Friesland'
    df.loc[df['Municipality']=='Mill en Sint Hubert', 'Province']='North Brabant'
    df.loc[df['Municipality']=='Hoeksche Waard', 'Province']='South Holland'
    df.loc[df['Municipality']=='Het Hogeland', 'Province']='Groningen'
    df.loc[df['Municipality']=='Bergen (NH.)', 'Province']=' North Holland'
    df.loc[df['Municipality']=='Bergen (L.)', 'Province']='Limburg'
    df.loc[df['Municipality']=='Beekdaelen', 'Province']='Limburg'
    df.loc[df['Municipality']=='Altena', 'Province']='North Brabant'
    df.loc[df['Municipality']=='Molenlanden', 'Province']='South Holland'
    df.loc[df['Municipality']=='Noardeast-FryslÃ¢n', 'Municipality']='Noardeast-Fryslan'
    df.loc[df['Municipality']=='Noardeast-Fryslan', 'Province']='Friesland'
    df.loc[df['Municipality']=='SÃºdwest-FryslÃ¢n', 'Municipality']='Sudwest-Fryslan'
    df.loc[df['Municipality']=='Sudwest-Fryslan', 'Province']='Friesland'
    df.Province=df.Province.str.lstrip()
    
    #Saving the data to csv file in a certian path
    df.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/rivm_data_3_prov_mun.csv',  index = True, header=True)
    
def rivm_data4():
    '''Age & gender distirbution of 
    deaths due to Covid-19 disease 
    from rivm.nl'''
    
    # Scrapping the data from rivm.nl
    request_data=requests.get('https://www.rivm.nl/coronavirus-covid-19/grafieken').text
    soup = BeautifulSoup(request_data,'html.parser')
    my_table=soup.find_all('script', {'type':"application/json"})
    my_table=str(my_table)
    my_table=my_table.split("csv")
    
    #Data Wrangling
    table_4 = my_table[1]
    table_4 = table_4.replace('\\u0022', '')
    table_4 = table_4.split("data:")[0]
    table_4 = str(table_4).split(']","pre')[0]
    table_4 = table_4.replace("]", '')
    table_4 = table_4.split("[")
    
    #Setting the date to a data frame & Cleaning the data 
    df4 = pd.DataFrame([sub.split(",") for sub in table_4])
    df4.drop([3], axis=1, inplace=True)
    df4.drop([0,1,2],inplace=True)
    df4.columns=["Age","Male","Female"]
    df4.Male = pd.to_numeric(df4.Male)
    df4.Female = pd.to_numeric(df4.Female)
    df4.loc[:,"Total"] = df4.Male + df4.Female
    df4['Country']='Netherlands'
    
    #Saving the data to csv file in a certian path
    df4.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/rivm_data_4_gender_age_NL.csv',  index = True, header=True)
    
def ecdc_data():

    '''COVID-19 Cases and Deaths 
    data for the Netherlands 
    through ECDC website -
    daily updated excel file'''

    #Scrapping excel data via BS4 from ECDC & Setting the data to a dataframe
    today = datetime.now()
    today = today.strftime('%Y-%m-%d')
    yesterday = datetime.now() - timedelta(days=1)
    yesterday = yesterday.strftime('%Y-%m-%d')

    try: 
        url=f'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{today}.xlsx'
        df_s = pd.read_excel(url, parse_date=[0],index_col=0)#Parsing date is for time series analysis  
        df_domain = pd.DataFrame(df_s)
    
    except:    
        url=f'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{yesterday}.xlsx'
        df_s = pd.read_excel(url, parse_date=[0],index_col=0)#Parsing date is for time series analysis  
        df_domain = pd.DataFrame(df_s)
        
    #Data Wrangling
    df_domain.columns = ['Day','Month','Year','Cases','Deaths', 'Countries_and_territories','GeoId','Country_Code','Population','Continent']
    df_domain.drop('Day',1,inplace=True)
    df_domain.drop('Month',1,inplace=True)
    df_domain.drop('Year',1,inplace=True)
    df_domain.drop('Countries_and_territories',1,inplace=True)
    df_domain.drop('Country_Code',1,inplace=True)
    df_domain.drop('Population',1,inplace=True)
    df_domain.drop('Continent',1, inplace=True)
    df_domain = df_domain[df_domain['GeoId'] == 'NL']
    df_domain.drop('GeoId',1,inplace=True)
    df_domain.drop(df_domain.tail(1).index,inplace=True) # drop last row 31-12-2019
    df_domain['Case_Fatality_Ratio'] =df_domain.Deaths/df_domain.Cases
    
    #Saving the data to csv file in a certian path
    df_domain.to_csv (r'/content/drive/My Drive/NLP proje/Our codings/Data Analysis/CSV/ECDC_data_case_deaths.csv',  index = True, header=True)


    
main()


Enter your authorization code:
··········
Mounted at drive
1
